In [1]:
from carbon.sdk import CarbonSDK, Tokens as T
#from carbon import CarbonOrderUI, CarbonSimulatorUI
from carbon.helpers.widgets import CheckboxManager, DropdownManager, PcSliderManager
from matplotlib import pyplot as plt
import numpy as np
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonSDK))
!node --version

CarbonSDK v0.9 (8/Mar/2023)
v18.14.2


# Taker Functionality [NB30]

_this Notebook gives an overview over the taker functionality; for more streamlined examples see the following notebooks_

In order to start the server, please run

    source /Volumes/Private/bin/sdkserver
    node server/sdkserver.mjs

In [2]:
SDK = CarbonSDK(disclaimer=False, verbose=True, Tokens=T)
pairs, pairs_s = SDK.pairs(inclstr=True)
SDK.version().get("msg")

[req0] method=post, url=http://localhost:3118/api/scall/pairs, params={}
[req0] method=get, url=http://localhost:3118/, params={}


'Carbon SDK v0.9 (7/Mar/2022)'

## Pair selection

In [3]:
try:
    pairs_sel = [s.strip() for s in pairs_s.split(",")]
    pairs_w()
except:
    pairs_w = DropdownManager(pairs_sel, defaultval="USDC/ETH")
    pairs_w()

Dropdown(index=1, options=(('USDC/USDT', 'USDC/USDT'), ('USDC/ETH', 'USDC/ETH'), ('BNT/USDT', 'BNT/USDT'), ('B…

In [4]:
try:
    inverse_w()
except:
    inverse_w = CheckboxManager(["inverse"], values=[1])
    inverse_w()

In [5]:
tknb,tknq = pairs_w.value.split("/")
if inverse_w.values[0]:
    tknb,tknq = tknq, tknb
pair = f"{tknb}/{tknq}"
pair

'ETH/USDC'

## Market information

In [7]:
m = SDK.mGetMarketByPair(pair=pair)
m

[req0] method=post, url=http://localhost:3118/api/scall/get_min_rate_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}
[req0] method=post, url=http://localhost:3118/api/scall/get_max_rate_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'}
[req0] method=post, url=http://localhost:3118/api/scall/get_min_rate_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}
[req0] method=post, url=http://localhost:3118/api/scall/get_max_rate_by_pair, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee'}
[req0] method=post, url=http://localhost:3118/api/scall/get_liquidity_by_pair, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0

CarbonSDK.MarketByPair(pair='ETH/USDC', bestBid=2330.3861458234296, bestAsk=2500.0, liqBid=5269359.684327, liqAsk=142.67636977847653, lastBid=799.9999998679182, lastAsk=3000.0000000000327)

## Trading examples

#### `mPrepareTrade`

In [8]:
approx_price = SDK.roundsd(m.mid,3)
amount_tknb = 1000
approx_price, amount_tknb

(2420.0, 1000)

In [9]:
r1 = SDK.mPrepareTrade(pair, SDK.SELL, tknb, amount_tknb/approx_price)
r1

[req0] method=post, url=http://localhost:3118/api/scall/get_trade_data, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'amount': '0.4132231404958678', 'tradeByTargetAmount': False, 'filter': []}


CarbonSDK.TradeData(pair='ETH/USDC', tkn='ETH', traderBuySell='SELL', amount=0.4132231404958678, sourceToken=SDKToken(token='ETH', address='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', decimals=18, name='ETH'), targetToken=SDKToken(token='USDC', address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', decimals=6, name='USDC'), tradeByTargetAmount=False, tradeActions=[{'strategyId': 27, 'amount': 413223140259990940}], actionsWei=[{'id': 27, 'input': 413223140259990940, 'output': 960219280}], actionsTokenRes=[{'id': 27, 'sourceAmount': 0.41322314025999096, 'targetAmount': 960.21928}], tradeData={'totalSourceAmount': 0.41322314025999096, 'totalTargetAmount': 958.778951, 'effectiveRate': 2320.245062744447})

In [10]:
r2 = SDK.mPrepareTrade(pair, SDK.BUY, tknb, amount_tknb/approx_price)
r2

[req0] method=post, url=http://localhost:3118/api/scall/get_trade_data, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'amount': '0.4132231404958678', 'tradeByTargetAmount': True, 'filter': []}


CarbonSDK.TradeData(pair='ETH/USDC', tkn='ETH', traderBuySell='BUY', amount=0.4132231404958678, sourceToken=SDKToken(token='USDC', address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', decimals=6, name='USDC'), targetToken=SDKToken(token='ETH', address='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', decimals=18, name='ETH'), tradeByTargetAmount=True, tradeActions=[{'strategyId': 26, 'amount': 413223140495867800}], actionsWei=[{'id': 26, 'input': 413223140495867800, 'output': 1033819069}], actionsTokenRes=[{'id': 26, 'sourceAmount': 1033.819069, 'targetAmount': 0.4132231404958678}], tradeData={'totalSourceAmount': 1035.372128, 'totalTargetAmount': 0.4132231404958678, 'effectiveRate': 0.0003991059149854455})

In [11]:
r3 = SDK.mPrepareTrade(pair, SDK.SELL, tknq, amount_tknb)
r3

[req0] method=post, url=http://localhost:3118/api/scall/get_trade_data, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'amount': '1000', 'tradeByTargetAmount': False, 'filter': []}


CarbonSDK.TradeData(pair='ETH/USDC', tkn='USDC', traderBuySell='SELL', amount=1000, sourceToken=SDKToken(token='USDC', address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', decimals=6, name='USDC'), targetToken=SDKToken(token='ETH', address='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', decimals=18, name='ETH'), tradeByTargetAmount=False, tradeActions=[{'strategyId': 26, 'amount': 1000000000}], actionsWei=[{'id': 26, 'input': 1000000000, 'output': 399715101557389183}], actionsTokenRes=[{'id': 26, 'sourceAmount': 1000.0, 'targetAmount': 0.3997151015573892}], tradeData={'totalSourceAmount': 1000.0, 'totalTargetAmount': 0.3991155289050531, 'effectiveRate': 0.0003991155289050531})

In [12]:
r4 = SDK.mPrepareTrade(pair, SDK.BUY, tknq, amount_tknb)
r4

[req0] method=post, url=http://localhost:3118/api/scall/get_trade_data, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'amount': '1000', 'tradeByTargetAmount': True, 'filter': []}


CarbonSDK.TradeData(pair='ETH/USDC', tkn='USDC', traderBuySell='BUY', amount=1000, sourceToken=SDKToken(token='ETH', address='0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', decimals=18, name='ETH'), targetToken=SDKToken(token='USDC', address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', decimals=6, name='USDC'), tradeByTargetAmount=True, tradeActions=[{'strategyId': 27, 'amount': 1000000000}], actionsWei=[{'id': 27, 'input': 1000000000, 'output': 430359468510485359}], actionsTokenRes=[{'id': 27, 'sourceAmount': 0.43035946851048534, 'targetAmount': 1000.0}], tradeData={'totalSourceAmount': 0.4310059774767004, 'totalTargetAmount': 1000.0, 'effectiveRate': 2320.153436976541})

#### `mComposeTradeTransaction`

In [13]:
tx1 = SDK.mComposeTradeTransaction(**r1.cttkwargs)
tx1

[req0] method=post, url=http://localhost:3118/api/scall/compose_trade_by_source_transaction, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'tradeActions': [{'strategyId': '27', 'amount': '413223140259990940'}], 'deadline': '4294967295', 'minReturn': '862.9010559000001', 'overrides': {'gasLimit': 999999999}}


{'data': '0xf1c5e014000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000000000000336ed33f0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000001b00000000000000000000000000000000000000000000000005bc1041b82b0d9c',
 'to': '0x2FdE9E0cE83769E88eAA9FB7BE27cb26c385d1b2',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'},
 'value': {'type': 'BigNumber', 'hex': '0x05bc1041b82b0d9c'}}

In [14]:
tx2 = SDK.mComposeTradeTransaction(**r2.cttkwargs)
tx2

[req0] method=post, url=http://localhost:3118/api/scall/compose_trade_by_target_transaction, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'tradeActions': [{'strategyId': '26', 'amount': '413223140495867800'}], 'deadline': '4294967295', 'maxInput': '1138.9093408', 'overrides': {'gasLimit': 999999999}}


{'data': '0x102ee9ba000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff0000000000000000000000000000000000000000000000000000000043e2609c0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000001a00000000000000000000000000000000000000000000000005bc1041c63a3f98',
 'to': '0x2FdE9E0cE83769E88eAA9FB7BE27cb26c385d1b2',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}}

In [15]:
tx3 = SDK.mComposeTradeTransaction(**r3.cttkwargs)
tx3

[req0] method=post, url=http://localhost:3118/api/scall/compose_trade_by_source_transaction, params={'sourceToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'targetToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'tradeActions': [{'strategyId': '26', 'amount': '1000000000'}], 'deadline': '4294967295', 'minReturn': '0.3592039760145478', 'overrides': {'gasLimit': 999999999}}


{'data': '0xf1c5e014000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000004fc261cf0414b580000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000001a000000000000000000000000000000000000000000000000000000003b9aca00',
 'to': '0x2FdE9E0cE83769E88eAA9FB7BE27cb26c385d1b2',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}}

In [16]:
tx4 = SDK.mComposeTradeTransaction(**r4.cttkwargs)
tx4

[req0] method=post, url=http://localhost:3118/api/scall/compose_trade_by_target_transaction, params={'sourceToken': '0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee', 'targetToken': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', 'tradeActions': [{'strategyId': '27', 'amount': '1000000000'}], 'deadline': '4294967295', 'maxInput': '0.47410657522437044', 'overrides': {'gasLimit': 999999999}}


{'data': '0x102ee9ba000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000006945d6b11ecf5080000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000001b000000000000000000000000000000000000000000000000000000003b9aca00',
 'to': '0x2FdE9E0cE83769E88eAA9FB7BE27cb26c385d1b2',
 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'},
 'value': {'type': 'BigNumber', 'hex': '0x3b9aca00'}}

#### Submit transactions

In [17]:
SDK.signsubmittx(tx1, sign=True)

[req0] method=post, url=http://localhost:3118/api/signsubmittx, params={'tx': {'data': '0xf1c5e014000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000000000000336ed33f0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000001b00000000000000000000000000000000000000000000000005bc1041b82b0d9c', 'to': '0x2FdE9E0cE83769E88eAA9FB7BE27cb26c385d1b2', 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}, 'value': {'type': 'BigNumber', 'hex': '0x05bc1041b82b0d9c'}}, True: True}


{'txReceipt': {'hash': '0xcc7f30f71b5644832af2fbc240d6e47c71f2c6beb86cc6d5c0ec4fee9c74c655',
  'nonce': None,
  'gasLimit': None,
  'gasPrice': None,
  'data': None,
  'value': None,
  'chainId': None,
  'confirmations': 0,
  'from': None}}

In [18]:
SDK.signsubmittx(tx2, sign=True)

[req0] method=post, url=http://localhost:3118/api/signsubmittx, params={'tx': {'data': '0x102ee9ba000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff0000000000000000000000000000000000000000000000000000000043e2609c0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000001a00000000000000000000000000000000000000000000000005bc1041c63a3f98', 'to': '0x2FdE9E0cE83769E88eAA9FB7BE27cb26c385d1b2', 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}}, True: True}


{'txReceipt': {'hash': '0x50225d77b7f7b4b90280efdbdf0040a32acf0d60337ce1d43fe039128311746f',
  'nonce': None,
  'gasLimit': None,
  'gasPrice': None,
  'data': None,
  'value': None,
  'chainId': None,
  'confirmations': 0,
  'from': None}}

In [19]:
SDK.signsubmittx(tx3, sign=True)

[req0] method=post, url=http://localhost:3118/api/signsubmittx, params={'tx': {'data': '0xf1c5e014000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000004fc261cf0414b580000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000001a000000000000000000000000000000000000000000000000000000003b9aca00', 'to': '0x2FdE9E0cE83769E88eAA9FB7BE27cb26c385d1b2', 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}}, True: True}


{'txReceipt': {'hash': '0x6f948d5a24980c693c16b8df1df4c670252fd7055f8bd2a399462f9b29f0010a',
  'nonce': None,
  'gasLimit': None,
  'gasPrice': None,
  'data': None,
  'value': None,
  'chainId': None,
  'confirmations': 0,
  'from': None}}

In [21]:
SDK.signsubmittx(tx4, sign=True)

[req0] method=post, url=http://localhost:3118/api/signsubmittx, params={'tx': {'data': '0x102ee9ba000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb4800000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000ffffffff00000000000000000000000000000000000000000000000006945d6b11ecf5080000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000001b000000000000000000000000000000000000000000000000000000003b9aca00', 'to': '0x2FdE9E0cE83769E88eAA9FB7BE27cb26c385d1b2', 'gasLimit': {'type': 'BigNumber', 'hex': '0x3b9ac9ff'}, 'value': {'type': 'BigNumber', 'hex': '0x3b9aca00'}}, True: True}


{'txReceipt': {'hash': '0xbe7e3500a361d878804a13d9bacc8e6540a2123e6b9a153638b13d66368ba621',
  'nonce': None,
  'gasLimit': None,
  'gasPrice': None,
  'data': None,
  'value': None,
  'chainId': None,
  'confirmations': 0,
  'from': None}}